In [1]:
# We import dataclass to represent 1 gene
from dataclasses import dataclass
import random
import pandas as pd
import numpy as np
from deap import base, creator, tools, algorithms
import csv

In [2]:
# A gene contains 2 attributes : the food item's name and the number of servings of this item
@dataclass
class Gene:
    food_item_name: str
    serving: int

In [3]:
# We make a list representing all the meals we have in 1 chromosome 
Meals =["Breakfast", "Snack 1", "Lunch", "Snack 2", "Dinner"]
# We make a list representing all the food groups we have in 1 meal
Food_Groups =["Vegetables", "Fruits", "Grains", "Protein", "Dairy", "Fats and Oils"]

In [4]:
import csv

def load_food_data(csv_path):
    food_data = {}
    meal_flags = {}

    with open(csv_path, newline='', encoding='utf-8') as f:
        reader = csv.DictReader(f)

        for row in reader:
            group = row['Category']
            food = row['Food Item']

            if group not in food_data:
                food_data[group] = {}

            # Safely convert numeric columns, use 0 if empty
            def safe_float(x):
                try:
                    return float(x)
                except (ValueError, TypeError):
                    return 0.0

            food_data[group][food] = {
                'p': safe_float(row['Protein (g)']),
                'f': safe_float(row['Fats (g)']),
                'c': safe_float(row['Carbohydrates (g)']),
                'cal': safe_float(row['Calories (kcal)'])
            }

            # Safely convert meal flags (0 if empty)
            def safe_int(x):
                try:
                    return int(x)
                except (ValueError, TypeError):
                    return 0

            meal_flags[food] = {
                'Breakfast': safe_int(row['Breakfast']),
                'Snack 1': safe_int(row['Snack 1']),
                'Lunch': safe_int(row['Lunch']),
                'Snack 2': safe_int(row['Snack 2']),
                'Dinner': safe_int(row['Dinner'])
            }

    return food_data, meal_flags

FOOD_DATA, MEAL_FLAGS = load_food_data("Processed_Bahrain_Food_Dataset.csv")



In [5]:
# I created a dictionary that contains all the saudi guideline's servings
SERVING_GUIDELINES_BY_KCAL = {
    1200: {
        "Breakfast": {"Grains": 2, "Dairy": 1, "Protein": 1, "Vegetables": 1, "Fruits": 0, "Fats and Oils": 1},
        "Snack 1":    {"Grains": 0, "Dairy": 0, "Protein": 0, "Vegetables": 0, "Fruits": 1, "Fats and Oils": 0},
        "Lunch":     {"Grains": 2, "Dairy": 1, "Protein": 1, "Vegetables": 2, "Fruits": 0, "Fats and Oils": 0},
        "Snack 2":    {"Grains": 0, "Dairy": 0, "Protein": 0, "Vegetables": 0, "Fruits": 1, "Fats and Oils": 0},
        "Dinner":    {"Grains": 2, "Dairy": 0, "Protein": 1, "Vegetables": 1, "Fruits": 0, "Fats and Oils": 0}
    },

    1300: {
        "Breakfast": {"Grains": 2, "Dairy": 1, "Protein": 1, "Vegetables": 1, "Fruits": 0, "Fats and Oils": 1},
        "Snack 1":    {"Grains": 0, "Dairy": 0, "Protein": 0, "Vegetables": 0, "Fruits": 1, "Fats and Oils": 0},
        "Lunch":     {"Grains": 3, "Dairy": 1, "Protein": 1, "Vegetables": 1, "Fruits": 0, "Fats and Oils": 1},
        "Snack 2":    {"Grains": 0, "Dairy": 0, "Protein": 0, "Vegetables": 0, "Fruits": 1, "Fats and Oils": 0},
        "Dinner":    {"Grains": 2, "Dairy": 0, "Protein": 1, "Vegetables": 1, "Fruits": 0, "Fats and Oils": 0}
    },

    1400: {
        "Breakfast": {"Grains": 2, "Dairy": 1, "Protein": 1, "Vegetables": 1, "Fruits": 0, "Fats and Oils": 1},
        "Snack 1":    {"Grains": 0, "Dairy": 0, "Protein": 0, "Vegetables": 0, "Fruits": 1, "Fats and Oils": 0},
        "Lunch":     {"Grains": 3, "Dairy": 1, "Protein": 1, "Vegetables": 2, "Fruits": 0, "Fats and Oils": 1},
        "Snack 2":    {"Grains": 1, "Dairy": 0, "Protein": 0, "Vegetables": 0, "Fruits": 1, "Fats and Oils": 0},
        "Dinner":    {"Grains": 2, "Dairy": 0, "Protein": 1, "Vegetables": 1, "Fruits": 0, "Fats and Oils": 0}
    },

    1500: {
        "Breakfast": {"Grains": 2, "Dairy": 1, "Protein": 1, "Vegetables": 1, "Fruits": 0, "Fats and Oils": 1},
        "Snack 1":    {"Grains": 0, "Dairy": 0, "Protein": 0, "Vegetables": 0, "Fruits": 1, "Fats and Oils": 0},
        "Lunch":     {"Grains": 3, "Dairy": 1, "Protein": 2, "Vegetables": 1, "Fruits": 0, "Fats and Oils": 1},
        "Snack 2":    {"Grains": 1, "Dairy": 0, "Protein": 0, "Vegetables": 0, "Fruits": 1, "Fats and Oils": 0},
        "Dinner":    {"Grains": 2, "Dairy": 0, "Protein": 1, "Vegetables": 1, "Fruits": 1, "Fats and Oils": 0}
    },

    1600: {
        "Breakfast": {"Grains": 2, "Dairy": 1, "Protein": 1, "Vegetables": 1, "Fruits": 0, "Fats and Oils": 1},
        "Snack 1":    {"Grains": 1, "Dairy": 0, "Protein": 0, "Vegetables": 0, "Fruits": 1, "Fats and Oils": 0},
        "Lunch":     {"Grains": 3, "Dairy": 1, "Protein": 2, "Vegetables": 1, "Fruits": 0, "Fats and Oils": 1},
        "Snack 2":    {"Grains": 1, "Dairy": 0, "Protein": 0, "Vegetables": 0, "Fruits": 1, "Fats and Oils": 0},
        "Dinner":    {"Grains": 2, "Dairy": 0, "Protein": 1, "Vegetables": 1, "Fruits": 1, "Fats and Oils": 0}
    },

    1700: {
        "Breakfast": {"Grains": 3, "Dairy": 1, "Protein": 1, "Vegetables": 1, "Fruits": 0, "Fats and Oils": 1},
        "Snack 1":    {"Grains": 1, "Dairy": 0, "Protein": 0, "Vegetables": 0, "Fruits": 1, "Fats and Oils": 0},
        "Lunch":     {"Grains": 3, "Dairy": 1, "Protein": 3, "Vegetables": 2, "Fruits": 0, "Fats and Oils": 1},
        "Snack 2":    {"Grains": 1, "Dairy": 0, "Protein": 0, "Vegetables": 0, "Fruits": 1, "Fats and Oils": 0},
        "Dinner":    {"Grains": 2, "Dairy": 0, "Protein": 1, "Vegetables": 1, "Fruits": 1, "Fats and Oils": 0}
    },

    1800: {
        "Breakfast": {"Grains": 3, "Dairy": 1, "Protein": 2, "Vegetables": 1, "Fruits": 0, "Fats and Oils": 1},
        "Snack 1":    {"Grains": 1, "Dairy": 0, "Protein": 0, "Vegetables": 0, "Fruits": 1, "Fats and Oils": 0},
        "Lunch":     {"Grains": 3, "Dairy": 1, "Protein": 3, "Vegetables": 1, "Fruits": 0, "Fats and Oils": 1},
        "Snack 2":    {"Grains": 1, "Dairy": 0, "Protein": 0, "Vegetables": 0, "Fruits": 1, "Fats and Oils": 0},
        "Dinner":    {"Grains": 2, "Dairy": 0, "Protein": 1, "Vegetables": 1, "Fruits": 1, "Fats and Oils": 1}
    },

    1900: {
        "Breakfast": {"Grains": 3, "Dairy": 1, "Protein": 2, "Vegetables": 1, "Fruits": 0, "Fats and Oils": 1},
        "Snack 1":    {"Grains": 1, "Dairy": 0, "Protein": 0, "Vegetables": 0, "Fruits": 1, "Fats and Oils": 0},
        "Lunch":     {"Grains": 3, "Dairy": 1, "Protein": 3, "Vegetables": 2, "Fruits": 0, "Fats and Oils": 1},
        "Snack 2":    {"Grains": 1, "Dairy": 0, "Protein": 0, "Vegetables": 0, "Fruits": 1, "Fats and Oils": 0},
        "Dinner":    {"Grains": 2, "Dairy": 0, "Protein": 1, "Vegetables": 1, "Fruits": 1, "Fats and Oils": 1}
    },

    2000: {
        "Breakfast": {"Grains": 3, "Dairy": 1, "Protein": 2, "Vegetables": 1, "Fruits": 0, "Fats and Oils": 1},
        "Snack 1":    {"Grains": 1, "Dairy": 0, "Protein": 0, "Vegetables": 0, "Fruits": 1, "Fats and Oils": 0},
        "Lunch":     {"Grains": 3, "Dairy": 1, "Protein": 3, "Vegetables": 1, "Fruits": 0, "Fats and Oils": 1},
        "Snack 2":    {"Grains": 2, "Dairy": 0, "Protein": 0, "Vegetables": 0, "Fruits": 1, "Fats and Oils": 0},
        "Dinner":    {"Grains": 2, "Dairy": 0, "Protein": 2, "Vegetables": 1, "Fruits": 1, "Fats and Oils": 0}
    },

    2200: {
        "Breakfast": {"Grains": 3, "Dairy": 1, "Protein": 3, "Vegetables": 1, "Fruits": 0, "Fats and Oils": 1},
        "Snack 1":    {"Grains": 1, "Dairy": 0, "Protein": 0, "Vegetables": 0, "Fruits": 1, "Fats and Oils": 0},
        "Lunch":     {"Grains": 3, "Dairy": 1, "Protein": 3, "Vegetables": 2, "Fruits": 0, "Fats and Oils": 1},
        "Snack 2":    {"Grains": 2, "Dairy": 0, "Protein": 0, "Vegetables": 0, "Fruits": 1, "Fats and Oils": 0},
        "Dinner":    {"Grains": 2, "Dairy": 0, "Protein": 2, "Vegetables": 1, "Fruits": 1, "Fats and Oils": 1}
    }
}

In [6]:
"""
FOOD_DATA = {
    "Vegetables": {
        "Green Beans": {"p": 2.0, "f": 0.1, "c": 7.8, "cal": 34.1},
        "Potato": {"p": 4.3, "f": 0.2, "c": 37.2, "cal": 164.0},
        "Broccoli": {"p": 2.8, "f": 0.4, "c": 7.0, "cal": 55.0},
        "Carrots": {"p": 0.9, "f": 0.2, "c": 10.0, "cal": 41.0},
        "Spinach": {"p": 2.9, "f": 0.4, "c": 3.6, "cal": 23.0},
        "Tomato": {"p": 0.9, "f": 0.2, "c": 3.9, "cal": 18.0},
        "Bell Pepper": {"p": 1.0, "f": 0.3, "c": 6.0, "cal": 31.0},
        "Cauliflower": {"p": 1.9, "f": 0.3, "c": 5.0, "cal": 25.0},
        "Zucchini": {"p": 1.2, "f": 0.3, "c": 3.1, "cal": 17.0},
        "Cucumber": {"p": 0.7, "f": 0.1, "c": 3.6, "cal": 16.0},
    },

    "Fruits": {
        "Blueberries": {"p": 1.1, "f": 0.5, "c": 21.0, "cal": 82.6},
        "Apple": {"p": 0.4, "f": 0.2, "c": 19.1, "cal": 71.8},
        "Banana": {"p": 1.3, "f": 0.4, "c": 27.0, "cal": 105.0},
        "Orange": {"p": 0.9, "f": 0.1, "c": 12.0, "cal": 47.0},
        "Strawberries": {"p": 0.7, "f": 0.3, "c": 7.7, "cal": 32.0},
        "Grapes": {"p": 0.7, "f": 0.2, "c": 18.0, "cal": 69.0},
        "Watermelon": {"p": 0.6, "f": 0.2, "c": 7.6, "cal": 30.0},
        "Pineapple": {"p": 0.5, "f": 0.1, "c": 13.0, "cal": 50.0},
        "Mango": {"p": 0.8, "f": 0.4, "c": 15.0, "cal": 60.0},
        "Peach": {"p": 0.9, "f": 0.3, "c": 9.5, "cal": 39.0},
    },

    "Grains": {
        "Rice (cooked)": {"p": 4.25, "f": 0.44, "c": 44.51, "cal": 205.4},
        "Salted Cracker": {"p": 0.22, "f": 0.76, "c": 1.83, "cal": 15.06},
        "Whole Wheat Bread": {"p": 3.6, "f": 1.0, "c": 12.0, "cal": 69.0},
        "Oatmeal": {"p": 2.5, "f": 1.4, "c": 12.0, "cal": 68.0},
        "Quinoa (cooked)": {"p": 4.4, "f": 1.9, "c": 21.3, "cal": 120.0},
        "Pasta (cooked)": {"p": 5.0, "f": 0.9, "c": 25.0, "cal": 131.0},
        "Brown Rice": {"p": 2.6, "f": 0.9, "c": 23.0, "cal": 112.0},
        "Corn Tortilla": {"p": 1.4, "f": 0.7, "c": 10.7, "cal": 52.0},
        "Bagel": {"p": 3.0, "f": 0.5, "c": 14.0, "cal": 72.0},
        "Pita Bread": {"p": 2.7, "f": 0.7, "c": 16.8, "cal": 83.0},
    },

    "Protein": {
        "Omelet": {"p": 6.5, "f": 7.3, "c": 0.4, "cal": 95.8},
        "Canned Chicken": {"p": 30.9, "f": 11.3, "c": 0.0, "cal": 234.3},
        "Grilled Salmon": {"p": 25.0, "f": 13.0, "c": 0.0, "cal": 206.0},
        "Turkey Breast": {"p": 30.0, "f": 1.0, "c": 0.0, "cal": 135.0},
        "Ground Beef (lean)": {"p": 26.0, "f": 15.0, "c": 0.0, "cal": 250.0},
        "Tofu": {"p": 8.0, "f": 4.8, "c": 1.9, "cal": 76.0},
        "Black Beans": {"p": 8.9, "f": 0.5, "c": 23.7, "cal": 132.0},
        "Tuna (canned)": {"p": 26.0, "f": 1.0, "c": 0.0, "cal": 116.0},
        "Chicken Breast": {"p": 31.0, "f": 3.6, "c": 0.0, "cal": 165.0},
        "Eggs (boiled)": {"p": 6.3, "f": 5.3, "c": 0.6, "cal": 78.0},
    },

    "Dairy": {
        "Grated Parmesan": {"p": 1.9, "f": 1.4, "c": 0.2, "cal": 21.6},
        "Cheddar Cheese": {"p": 7.0, "f": 9.3, "c": 0.4, "cal": 112.8},
        "Greek Yogurt": {"p": 10.0, "f": 0.4, "c": 3.6, "cal": 59.0},
        "Milk (2%)": {"p": 8.0, "f": 5.0, "c": 12.0, "cal": 122.0},
        "Cottage Cheese": {"p": 11.0, "f": 2.3, "c": 3.4, "cal": 81.0},
        "Mozzarella": {"p": 6.3, "f": 6.0, "c": 0.6, "cal": 85.0},
        "Swiss Cheese": {"p": 8.0, "f": 7.8, "c": 1.5, "cal": 106.0},
        "Feta Cheese": {"p": 4.0, "f": 6.0, "c": 1.2, "cal": 75.0},
        "Ricotta Cheese": {"p": 3.2, "f": 2.4, "c": 1.9, "cal": 39.0},
        "Sour Cream": {"p": 0.5, "f": 2.9, "c": 0.6, "cal": 29.0},
    },

    "Fats and Oils": {
        "Corn Oil": {"p": 0.0, "f": 13.64, "c": 0.0, "cal": 120.55},
        "Mayonnaise": {"p": 0.13, "f": 4.91, "c": 3.51, "cal": 57.29},
        "Olive Oil": {"p": 0.0, "f": 14.0, "c": 0.0, "cal": 119.0},
        "Butter": {"p": 0.1, "f": 11.5, "c": 0.0, "cal": 102.0},
        "Avocado": {"p": 2.0, "f": 15.0, "c": 9.0, "cal": 160.0},
        "Peanut Butter": {"p": 4.0, "f": 8.0, "c": 3.5, "cal": 94.0},
        "Almonds": {"p": 6.0, "f": 14.0, "c": 6.0, "cal": 164.0},
        "Coconut Oil": {"p": 0.0, "f": 13.5, "c": 0.0, "cal": 117.0},
        "Sunflower Oil": {"p": 0.0, "f": 13.6, "c": 0.0, "cal": 120.0},
        "Walnuts": {"p": 4.3, "f": 18.5, "c": 3.9, "cal": 185.0},
    }
}
"""

'\nFOOD_DATA = {\n    "Vegetables": {\n        "Green Beans": {"p": 2.0, "f": 0.1, "c": 7.8, "cal": 34.1},\n        "Potato": {"p": 4.3, "f": 0.2, "c": 37.2, "cal": 164.0},\n        "Broccoli": {"p": 2.8, "f": 0.4, "c": 7.0, "cal": 55.0},\n        "Carrots": {"p": 0.9, "f": 0.2, "c": 10.0, "cal": 41.0},\n        "Spinach": {"p": 2.9, "f": 0.4, "c": 3.6, "cal": 23.0},\n        "Tomato": {"p": 0.9, "f": 0.2, "c": 3.9, "cal": 18.0},\n        "Bell Pepper": {"p": 1.0, "f": 0.3, "c": 6.0, "cal": 31.0},\n        "Cauliflower": {"p": 1.9, "f": 0.3, "c": 5.0, "cal": 25.0},\n        "Zucchini": {"p": 1.2, "f": 0.3, "c": 3.1, "cal": 17.0},\n        "Cucumber": {"p": 0.7, "f": 0.1, "c": 3.6, "cal": 16.0},\n    },\n\n    "Fruits": {\n        "Blueberries": {"p": 1.1, "f": 0.5, "c": 21.0, "cal": 82.6},\n        "Apple": {"p": 0.4, "f": 0.2, "c": 19.1, "cal": 71.8},\n        "Banana": {"p": 1.3, "f": 0.4, "c": 27.0, "cal": 105.0},\n        "Orange": {"p": 0.9, "f": 0.1, "c": 12.0, "cal": 47.0},\n   

In [7]:
# DEAP Configuration
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()

def create_individual():
    # Initializes 30 genes following 1600 kcal
    genes = []
    for meal in Meals:
        for group in Food_Groups:
            # Get the fixed serving count for this meal/group
            serving_count = SERVING_GUIDELINES_BY_KCAL[TARGET_KCAL][meal][group]

           # Filter foods allowed for this meal
            allowed_foods = [
                food for food in FOOD_DATA[group].keys() if MEAL_FLAGS[food][meal] == 1
            ]
            if not allowed_foods:
                allowed_foods = list(FOOD_DATA[group].keys())  # fallback

            # Randomly select food from allowed foods
            food_name = random.choice(allowed_foods)

            # Create gene with guideline serving count
            genes.append(Gene(food_item_name=food_name, serving=serving_count))

    return creator.Individual(genes)

toolbox.register("individual", create_individual)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# Mutation
def mutate_meal_plan(individual):
    mutations_done = 0
    all_indices = list(range(len(individual))) # List of all possible indices in the chromosome
    random.shuffle(all_indices) # Shuffle to pick randomly without repeating the same one

    for i in all_indices:
        if mutations_done >= 2:
            break  # Stop once we have mutated 2 genes

        # Check if the serving size is not zero
        if individual[i].serving > 0:
            # Find which group it belongs to (Vegetables, Grains, ...)
            group_name = Food_Groups[i % 6]
            # Pick a new food name from food group dictionary, but the serving remains the same
            individual[i].food_item_name = random.choice(list(FOOD_DATA[group_name].keys()))
            mutations_done += 1
    return individual,

def cx_meal_plan(parent1, parent2):
    GROUPS = 6
    MEALS = 5

    # Choose 2 meal cut points
    m1, m2 = sorted(random.sample(range(1, MEALS), 2))
    cut1, cut2 = m1 * GROUPS, m2 * GROUPS

    # Swap full meals
    parent1[cut1:cut2], parent2[cut1:cut2] = parent2[cut1:cut2], parent1[cut1:cut2]

    return parent1, parent2

def evaluate_meal_plan(individual):
    total_p = 0.0 # Protein grams
    total_f = 0.0 # Fat grams
    total_c = 0.0 # Carbohydrates grams
    total_cal = 0.0 # Total Calories

    # First: Sum up all grams and calories based on food item and serving count
    for gene in individual:
        group_name = None
        # Find which group this food belongs to
        for group, foods in FOOD_DATA.items():
            if gene.food_item_name in foods:
                group_name = group
                break

        if group_name:
            stats = FOOD_DATA[group_name][gene.food_item_name]
            total_p += stats['p'] * gene.serving
            total_f += stats['f'] * gene.serving
            total_c += stats['c'] * gene.serving
            total_cal += stats['cal'] * gene.serving

    # Second : Calculate actual macro percentages (p_c, p_p, p_f)
    g_total = total_p + total_f + total_c
    if g_total == 0: return 0, # Avoid division by zero

    p_p = total_p / g_total
    p_f = total_f / g_total
    p_c = total_c / g_total

    # Third :Target Ratios (The AMDR values)
    r_p, r_f, r_c = 0.20, 0.25, 0.55
    # Fourth: Calculate J_macro (Squared Error)
    j_macro = (p_c - r_c)**2 + (p_p - r_p)**2 + (p_f - r_f)**2

    # Fifth: Calculate J_cal
    r = (total_cal - TARGET_KCAL) / TARGET_KCAL
    j_cal = min(1, r**2)

    # Sixth: apply the variety constraint
    N = 30 # total number of food items in the chromosome ( N is taken from the report)
    # count food appearance
    food_appearance={}
    for gene in individual:
        food_appearance[gene.food_item_name]= food_appearance.get(gene.food_item_name, 0) + 1

    # Calculate the penalty for variety
    j_var = 0
    for food_name,count in food_appearance.items():
        # we penalize if the food item appears more than once
        if count > 1:
            j_var += max(0,count-1)**2
    # Normalize the penalty
    j_var = j_var / ((N-1)**2)


    # Seventh: calculate the score
    w_macro, w_var, w_cal = 0.33, 0.33, 0.33 #change to 0.25 when adding preference
    score = w_macro * j_macro + w_var * j_var + w_cal * j_cal
    # store the penalties to print later
    individual.j_var= j_var
    individual.j_macro = j_macro
    individual.j_cal = j_cal

    # Lastly: We want to minimize score, so we return a fitness that increases as score decreases
    return 1 / (1 + score),


toolbox.register("evaluate", evaluate_meal_plan)
toolbox.register("mate", cx_meal_plan) # Two-point crossover
toolbox.register("select", tools.selTournament, tournsize=3) # Tournament selection
toolbox.register("mutate", mutate_meal_plan)

In [8]:
# Execution Loop
def run_simulation(target_kcal):
    global TARGET_KCAL
    TARGET_KCAL = target_kcal   # tells create_individual which guideline to use

    pop = toolbox.population(n=50) # Population = 50

    hof = tools.HallOfFame(1) # Keeps the absolute best found across all generations

    mu = len(pop) # Number of individuals to select for the next generation
    lambd = 100 # Number of children to produce in each generation

    # Statistics object
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", np.mean)
    stats.register("min", np.min)
    stats.register("max", np.max)

    # Run Elitism-Based GA
    algorithms.eaMuPlusLambda(pop, toolbox,
                               mu=mu,
                               lambda_=lambd,
                               cxpb=0.7,   # Crossover probability
                               mutpb=0.2,  # Mutation probability
                               ngen=20,    # Number of generations
                               stats=stats,
                               halloffame=hof,
                               verbose=True)

    return hof[0]

In [9]:
# Execute the Simulation
best_plan = run_simulation(1600)

# Calculate Final Nutritional Totals
total_p, total_f, total_c, total_cal = 0.0, 0.0, 0.0, 0.0

print(f"\n{'MEAL':<12} | {'GROUP':<14} | {'FOOD ITEM':<25} | {'SERV.':<6} | {'P (g)':<6} | {'F (g)':<6} | {'C (g)':<6} | {'CALS':<6}")
print("-" * 75)

for i, gene in enumerate(best_plan):
    meal = Meals[i // 6]
    group = Food_Groups[i % 6]

    # Get food stats from dataset
    stats = FOOD_DATA[group][gene.food_item_name]

    # Calculate totals for this gene
    p = stats['p'] * gene.serving
    f = stats['f'] * gene.serving
    c = stats['c'] * gene.serving
    cal = stats['cal'] * gene.serving

    # Update running totals
    total_p += p
    total_f += f
    total_c += c
    total_cal += cal

    # Print row (only if servings > 0 for clarity)
    if gene.serving > 0:
        print(f"{meal:<12} | {group:<14} | {gene.food_item_name:<25} | {gene.serving:<6} | {p:<6.1f} | {f:<6.1f} | {c:<6.1f} | {cal:<6.1f}")

# Calculate Final Percentages
g_total = total_p + total_f + total_c
p_p = (total_p / g_total) * 100
p_f = (total_f / g_total) * 100
p_c = (total_c / g_total) * 100
variety= (1 - best_plan.j_var) * 100
calories = (total_cal / TARGET_KCAL) * 100

# Print Summary Report
print("\n" + "="*40)
print("       DAILY NUTRITION SUMMARY")
print("="*40)
print(f"Total Energy:   {total_cal:.1f} kcal")
print(f"Total Weight:   {g_total:.1f} g (Macro Grams)")
print("-" * 40)
print(f"Nutrient     | Actual % | Target %")
print(f"Carbs (c)    | {p_c:>7.1f}% | 55.0%")
print(f"Protein (p)  | {p_p:>7.1f}% | 20.0%")
print(f"Fats (f)     | {p_f:>7.1f}% | 25.0%")
print(f"Meal variety | {variety:>7.1f}% | 100%")
print(f"Meal calorie | {calories:>7.1f}% | 100%")
print("="*40)

gen	nevals	avg     	min     	max     
0  	50    	0.830539	0.741453	0.971904
1  	86    	0.913023	0.770529	0.971904
2  	91    	0.946699	0.878483	0.988903
3  	85    	0.966402	0.928836	0.988903
4  	91    	0.979851	0.958052	0.993732
5  	91    	0.986182	0.973909	0.993732
6  	93    	0.989467	0.983584	0.994586
7  	93    	0.992914	0.988903	0.995387
8  	92    	0.994165	0.991868	0.996386
9  	93    	0.995093	0.993332	0.996386
10 	87    	0.995558	0.993914	0.996847
11 	92    	0.995812	0.994621	0.996847
12 	95    	0.996225	0.995102	0.997589
13 	91    	0.996498	0.996165	0.997589
14 	90    	0.996845	0.996342	0.997613
15 	88    	0.997249	0.996386	0.997646
16 	89    	0.997525	0.99661 	0.997977
17 	85    	0.997653	0.997213	0.998008
18 	91    	0.997699	0.997589	0.997997
19 	88    	0.997789	0.99686 	0.998008
20 	89    	0.997867	0.997418	0.998178

MEAL         | GROUP          | FOOD ITEM                 | SERV.  | P (g)  | F (g)  | C (g)  | CALS  
------------------------------------------------------------